# Neo4j: Application of graph algorithms
COURSE: INFO9016-1 Advanced Databases

AUTHORS: 
- Emilien DE LA BRASSINNE BONARDAUX 
- Yanis GEURTS
- Eri VAN DE VYVER


## Introduction

Graph databases are designed to represent data as nodes interconnected with edges, which respectively model entities and their relationships. These databases are based on the mathematical notion of graphs, resulting in a more flexible representation of data than relational databases. This representation allows designing algorithms more efficiently to answer different kinds of questions using graph properties and traversal methods from graph theory.[1]

In this tutorial, Neo4j has been chosen to illustrate the usage of graph databases, since it allows modeling and querying graph databases more easily. It uses the Cypher query language, which is optimised for pattern matching and graph exploration and combines it with built-in graph algorithms such as PageRank and shortest path. Unlike relational databases that rely on tables and JOIN operations, Neo4j uses a native graph structure that directly stores relationships, enabling faster and more scalable queries across networks.[2]

Neo4j is used for a wide range of real-world applications such as fraud detection, where it identifies suspicious patterns by analyzing transaction graphs or in recommendation systems, where it leverages similarity and collaborative filtering to suggest products or content.

This tutorial uses a dataset of airport routes [3] to show how graph databases handle tasks that are more difficult for traditional databases, especially when analyzing connectivity, hubs, and optimal paths across complex networks.

### Presentation of the dataset

The airport routes dataset showcases Neo4j’s real-world use case in transportation and logistics. It contains 3,503 airports and 46,389 flight routes. It enables getting information like shortest paths between cities and route optimization. This dataset is well suited for exploring network connectivity, route planning and airline performance analysis. 


## Setup

For following this tutorial, docker must already be installed. Once installed execute these lines in your terminal: 

`docker pull neo4j`

`docker compose up`

It will create a Neo4j container running the database service.

Install the Neo4j python driver and import next libraries.

In [49]:
!pip install neo4j
!pip install pandas
!pip install plotly
!pip install nbformat
from neo4j import GraphDatabase
import pandas as pd

Now that everything is set up, the actual python environment just need to connect to the Neo4j docker container.

In [50]:
# Define Connection Parameters
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "neo4j")  # Default credentials for Neo4j when no auth is set

# Establish Connection and Query Executor
try:
    driver = GraphDatabase.driver(URI, auth=AUTH)
    driver.verify_connectivity()
    print("Connection to Neo4j successful!")
except Exception as e:
    print(f"Connection failed. Error: {e}")

def run_cypher_query(query):
    with driver.session() as session:
        result = session.execute_write(lambda tx: tx.run(query).data())
        return result
def run_cypher_query_df(query, params={}):
    """
    Executes a Cypher query using the global 'driver'
    and returns the result as a pandas DataFrame.
    """
    try:
        df = driver.execute_query(
            query,
            parameters_=params,
            database_="neo4j", 
            result_transformer_=pd.DataFrame
        )
        return df
    
    except Exception as e:
        print(f"Query failed: {e}")
        return pd.DataFrame()

Connection to Neo4j successful!


Let's begin by creating our graph database. We first define **constraints** and **indexes**.  
These ensure data consistency, improve performance, and prevent the creation of duplicate nodes.

- **Uniqueness constraints** guarantee that key identifiers such as airport IATA codes or country codes appear only once.
- **Indexes** accelerate point lookups on frequently accessed properties. In this case, we have an index on the location property of airports.

Creating constraints before loading data is important: Neo4j enforces them during the import phase and prevents storing inconsistent or duplicate nodes.

The following code creates:

- 5 uniqueness constraints  
- 1 index on the `location` property of `Airport`

In [51]:
print("\nStep 1: Creating Constraints and Indexes")

CONSTRAINTS_AND_INDEXES = [
    "CREATE CONSTRAINT airports IF NOT EXISTS FOR (a:Airport) REQUIRE a.iata IS UNIQUE",
    "CREATE CONSTRAINT cities IF NOT EXISTS FOR (c:City) REQUIRE (c.name, c.country_code) IS UNIQUE",
    "CREATE CONSTRAINT regions IF NOT EXISTS FOR (r:Region) REQUIRE r.name IS UNIQUE",
    "CREATE CONSTRAINT countries IF NOT EXISTS FOR (c:Country) REQUIRE c.code IS UNIQUE",
    "CREATE CONSTRAINT continents IF NOT EXISTS FOR (c:Continent) REQUIRE c.code IS UNIQUE",
    "CREATE INDEX locations IF NOT EXISTS FOR (air:Airport) ON (air.location)"
]

try:
    for query in CONSTRAINTS_AND_INDEXES:
        run_cypher_query(query)
    print("Constraints and indexes created successfully.")

except Exception as e:
    print(f"Failed to create constraints/indexes: {e}")


Step 1: Creating Constraints and Indexes
Constraints and indexes created successfully.


Next, we import all nodes and their complete geo-hierarchy from the `airport-node-list.csv` file.  
This step builds the structure of our graph by creating:

- **Five node labels:** `Airport`, `City`, `Region`, `Country`, and `Continent`
- **Four relationship types:** `:IN_CITY`, `:IN_REGION`, `:IN_COUNTRY`, and `:ON_CONTINENT`

Each row in the CSV is used to:

1. **Create (or merge) the corresponding nodes**.  
2. **Link the nodes** using the relationships.  
3. **Assign all airport attributes** (IATA, ICAO, altitude, runways, coordinates, etc.) using the `SET` command.

This ensures that every airport is uniquely connected to its city, region, country, and continent while avoiding duplicate nodes across the dataset.

In [52]:
print("\nStep 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv")

NODE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///airport-node-list.csv' AS row

// Creation of the nodes 
MERGE (a:Airport {iata: row.iata})
MERGE (r:Region {name: row.region})
MERGE (co:Country {code: row.country})
MERGE (con:Continent {name: row.continent})
MERGE (ci:City {name: row.city, country_code: row.country})

// Creation of the relationship types
MERGE (a)-[:IN_CITY]->(ci)
MERGE (a)-[:IN_COUNTRY]->(co)
MERGE (ci)-[:IN_COUNTRY]->(co)
MERGE (r)-[:IN_COUNTRY]->(co)
MERGE (a)-[:IN_REGION]->(r)
MERGE (ci)-[:IN_REGION]->(r)
MERGE (a)-[:ON_CONTINENT]->(con)
MERGE (ci)-[:ON_CONTINENT]->(con)
MERGE (co)-[:ON_CONTINENT]->(con)
MERGE (r)-[:ON_CONTINENT]->(con)

// Assigning attributes
SET a.id = row.id,
    a.icao = row.icao,
    a.descr = row.descr,
    a.runways = toInteger(row.runways),
    a.longest = toInteger(row.longest),
    a.altitude = toInteger(row.altitude),
    a.latitude = toFloat(row.lat),
    a.longitude = toFloat(row.lon)
    
    // Removed any potential a.city property to keep city as a separate City label

RETURN count(a) AS airports_loaded
"""

results_nodes = run_cypher_query(NODE_LOADING_QUERY)
print(f"Airports and Geographies loaded. Total Airports: {results_nodes[0]['airports_loaded']}")


Step 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv
Airports and Geographies loaded. Total Airports: 3503


Finally, we import the `:HAS_ROUTE` relationships between airports.  
Each relationship connects a source airport to a destination airport and includes a `distance` property, representing the distance between the two airports.  

This distance will later be used to create **weighted graphs** for routing and network analysis.

In [53]:

print("\nStep 3: Loading Routes from iroutes-edges.csv")

ROUTE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///iroutes-edges.csv' AS row

MATCH (source:Airport {iata: row.src})
MATCH (target:Airport {iata: row.dest})

MERGE (source)-[r:HAS_ROUTE]->(target)
ON CREATE SET r.distance = toFloat(row.dist)

RETURN count(r) AS routes_loaded
"""

results_routes = run_cypher_query(ROUTE_LOADING_QUERY)
print(f"Routes loaded. Total HAS_ROUTE relationships: {results_routes[0]['routes_loaded']}")


Step 3: Loading Routes from iroutes-edges.csv
Routes loaded. Total HAS_ROUTE relationships: 46389


The structure of the dataset's labels and relationships can be seen in the figure below: 
![](Images/Structure.png "Structure of the dataset")

The previous code for the three steps of the setup is adapted from:

3. Neo4j. Airport Routes Graph Example. GitHub, url: https://github.com/neo4j-graph-examples/airport-routes. Accessed 14 Nov 2025.

## Features of Neo4j

Neo4j is a native graph database built around the idea that relationships are as important as the data itself.  

Neo4j follows a Property Graph Model, made of:

- Nodes: Entities in the graph. Example: `(:Airport {iata:"BRU"})`
- Relationships: Directed edges connecting nodes. Example:  `(:Airport)-[:HAS_ROUTE]->(:Airport)`
- Properties: Key–value pairs assigned to nodes or relationships. Example: `a.latitude = 50.5859`
- Labels: Group nodes into types/classes. Example: `:Airport`, `:City`
- Relationship types: Classify relationship semantics. Example: `:IN_CITY`, `:HAS_ROUTE`

### Cypher Query Language Basics

Cypher is Neo4j’s declarative graph query language. It allows users to express complex queries in a readable, SQL-like style for connected data. Here we will show how to perform various simple queries.

#### Count nodes
First, we can count nodes in the graph using the *MATCH* clause to retrieve all airports and return the total count.

In [54]:
query = """
MATCH (a:Airport)
RETURN count(a) AS number_of_airports;
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0
0,3503


#### Find airport
To find a specific node, we use the MATCH clause and specify the attribute we are searching for. Here, we seek Brussels Airport using the IATA code "BRU".

In [55]:
query = """
MATCH (a:Airport {iata: "BRU"}) RETURN a.descr, a.runways, a.altitude, a.longest, a.latitude, a.longitude
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0,1,2,3,4,5
0,Brussels Airport,3,184,11936,50.901402,4.48444


#### Find connected airports

We are searching for all airports that are connected to Brussels Airport. 

In Cypher, a relationship `r` from node `a` to node `b` is represented by `(a)-[r]->(b)`. 

In [56]:
query = """
MATCH (a1:Airport {iata: "BRU"})-[:HAS_ROUTE]->(a2:Airport)
                                 RETURN a2.iata, a2.descr
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0,1
0,NBO,"Nairobi, Jomo Kenyatta International Airport"
1,IKA,Imam Khomeini International Airport
2,CPH,Copenhagen Kastrup Airport
3,LIS,Lisbon Portela Airport
4,RUH,"Riyadh, King Khaled International Airport"
...,...,...
189,FNC,"Funchal, Madeira Airport"
190,ORD,Chicago O'Hare International Airport
191,KEF,"Reykjavik, Keflavik International Airport"
192,NCE,Nice-Cote d'Azur Airport


#### Filter on relationship
We will now identify, among all airports that have a direct route from Brussels Airport, which ones are farther than 2,000 miles.

A filter in cypher is done with the *WHERE* clause.

In [57]:
query = """
    MATCH (a1:Airport {iata: "BRU"})-[r:HAS_ROUTE]->(a2:Airport)
    WHERE r.distance > 2000
    RETURN a2.iata, a2.descr
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0,1
0,NBO,"Nairobi, Jomo Kenyatta International Airport"
1,IKA,Imam Khomeini International Airport
2,RUH,"Riyadh, King Khaled International Airport"
3,ATL,Hartsfield - Jackson Atlanta International Air...
4,EBB,Entebbe International Airport
5,EVN,Zvartnots International Airport
6,BJM,Bujumbura International Airport
7,NSI,Yaoundé Nsimalen International Airport
8,PUJ,Punta Cana International Airport
9,IAD,Washington Dulles International Airport


#### Find airports within three flights
We look for all airports that can be reached in three flights or fewer, starting from Brussels Airport. We can sort the result by using the clause *ORDER BY*

In [58]:
query = """
    MATCH a = (start:Airport {iata:"BRU"})-[:HAS_ROUTE*1..3]->(dest:Airport)
    RETURN DISTINCT dest.iata AS airport, length(a) AS num_flights
    ORDER BY num_flights, airport;
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0,1
0,ABJ,1
1,ACC,1
2,ACE,1
3,ADB,1
4,AGA,1
...,...,...
4842,ZUM,3
4843,ZVK,3
4844,ZYI,3
4845,ZYL,3


#### Data types in Neo4j 
Neo4j supports:
- Primitive types (Integer, Float, String, Boolean, Lists)
- Temporal types (date, datetime, duration)
- Spatial types: `Point`

The `Point` type is the mechanism implemented in Neo4j to store geographical data. It supports:

- latitude/longitude points
- cartesian points
- `distance()` between points
- indexing on points

[6]

First, here is how to retrieve the point property of Brussels Airport from the dataset.

In [59]:
query = """
MATCH (a:Airport {iata:"BRU"})
RETURN point({latitude:a.latitude, longitude:a.longitude}) AS geo_point;
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0
0,"(4.48443984985, 50.9014015198)"


The distance between Brussels Aiport and Brussels South Charleroi Airport can be computed as followed.

In [60]:
query = """
MATCH (a:Airport {iata:"BRU"}), (b:Airport {iata:"CRL"})
RETURN point.distance(
  point({latitude:a.latitude, longitude:a.longitude}),
  point({latitude:b.latitude, longitude:b.longitude})
) / 1000 AS distance_km;
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0
0,49.272828


Creating a spatial index 

In [61]:
query = """
CREATE INDEX airport_location IF NOT EXISTS
FOR (a:Airport) ON (a.location);
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

""


Finding the nearest airports from a given place. Here we specify the place by giving the latitude and longitude.

In [62]:
query = """
WITH point({latitude:50.5859, longitude:5.56027}) AS montef
MATCH (a:Airport)
WITH a, point.distance(montef, point({latitude:a.latitude, longitude:a.longitude})) / 1000 AS dist
RETURN a.iata, dist
ORDER BY dist ASC
LIMIT 5;
"""

with driver.session() as session:
    result = run_cypher_query_df(query)
result

,0,1
0,LGG,10.061613
1,MST,39.164735
2,CRL,79.567205
3,BRU,83.525441
4,EIN,97.077585


### Other features

Neo4j also presents important features. It is ACID compliant. Neo4j supports full-text indexing, which allows efficient search across textual properties. It may be useful for matching airport descriptions or names. Finally, Neo4j has schema flexibility. Users *can* define constraints, add properties at any time and you don't need to predefine a full-schema.

### Advantages and limitations

Neo4j’s biggest strength compared to relational databases is how it handles connected queries. In a relational database, queries that require exploring several layers of relationships (such as “find all airports reachable within three flights from Brussels”) require multiple JOIN operations. As the number of JOINs increases, the performance degrades, since each JOIN requires matching rows across large tables and scanning indexes. Neo4j, however, uses a native graph storage where relationships are stored as direct pointers between nodes. This makes multi-hop queries scale with the length of the path rather than with the number of records, which is a fundamental advantage when working with network structures. This means that algorithms such as shortest paths are executed in constant time relative to the number of hops, rather than degrading with dataset size. This will be shown in the last question of this notebook where we are showing how to execute the A* algorithm to find the shortest flight path. 

Another strength of Neo4j is the built-in functions like PageRank or community detection that are optimized to run directly inside the database engine, avoiding the overhead of exporting data into external analytics tools. This makes Neo4j particularly advantageous for real-time applications such as fraud detection, recommendation engines and route optimization, where speed is important. 


Cypher allows users to express graph queries using pattern matching instead of JOINs. Many complex relational queries become simpler and more readable when expressed as graph patterns.


The main drawback is that for flat, tabular or purely transactional data, relational databases often perform better.


Finally, because Neo4j stores relationships explicitly, datasets with extremely dense connectivity can consume more storage compared to normalized relational schemas.


[4]

### Real-world use cases
Because of its advantages, Neo4j is widely adopted in domains where data is highly interconnected. Here are some examples of use cases[7].
- **Fraud Detection**: Traditional databases struggle to detect fraud patterns because they require slow JOIN operations to link indirect relationships. As Neo4j stores these relationships, it allows 1000 times faster query performance for detecting patterns. For example, Gilead Sciences uses Neo4j to combat huge pharmaceutical fraud [8].
- **Life sciences problems**: As life sciences problems are graph problems (e.g., genes, proteins, and drugs), neo4j is naturally optimized for it. It allows to represent biological, chemical, and clinical data intuitively as well as accelerate discovery. AstraZeneca uses Neo4j [9] to build knowledge graphs and it allows researchers to identify potential drug targets that would be invisible in standard tables. 
- **Supply chain**: Like for the previous use cases, the supply chain's best representation is done with graph representation. the US army uses Neo4j to be able to effectively manage and analyze over 5 billion data nodes for equipment maintenance [10]. 
- **Recommendation engine**: While traditional engines often rely on pre-calculated data, Neo4j enables real-time recommendations by traversing the graphs instantly. Companies like eBay used Neo4j to personalize content based on user history and immediate behavior [11].


## Running Graph Algorithms


### Using the GDS library

Typically, when one wants to use graph algorithms, the Graph Data Science library is the best option. Indeed, it provides efficiently implemented versions of graph algorithms. In addition, it offers an intuitive framework that connects with Neo4j's graph database [5]. 


In Cypher, the way to call a graph algorithm with GDS is as follows: 

`CALL gds[.<tier>].<algorithm>.<execution-mode>[.<estimate>](`

`graphName: String,`

  `configuration: Map)`

Where graphName is a projected graph keeping the attribute we need for our algorithms.

### Path Finding Algorithms

#### A* Algorithm

For this first example, we are going to apply the A* algorithm from the GDS library to find the shortest path between two cities. To accelerate search, it uses a heuristic. In our case, the heuristic computes the Haversine distance [6] (spherical distance) based on the latitude and longitude of the city. Because the original dataset contains airports connected by routes, we must first build a projected graph that contains only City nodes and City–City connections. 

A city is connected to another city if at least one airport in the first city has a route to an airport in the second city. 
The Cypher projection deriving this relationship is:

`(c1:City)<-[:IN_CITY]-(a1:Airport)-[:HAS_ROUTE]->(a2:Airport)-[:IN_CITY]->(c2:City)`


With the `RETURN` clause, it produces weighted edges where the weight corresponds to the flight distance.

As the cities don't currently have longitude and latitude properties, we have to set them. Here we choose to take the average of all the airports within a city.

In [63]:
query0 = """   
MATCH (c:City)<-[:IN_CITY]-(a:Airport)
WITH c, avg(a.latitude) AS lat, avg(a.longitude) AS lon
SET c.latitude = lat, c.longitude = lon
RETURN count(c) AS cities_updated
"""

with driver.session() as session:
    result = run_cypher_query_df(query0)
 
result

,0
0,3409


In [64]:
query1 = """   
    CALL gds.graph.exists('city_routes') YIELD exists
    WITH exists
    WHERE exists
    CALL gds.graph.drop('city_routes') YIELD graphName
    RETURN graphName
"""

query2 = """   
    CALL gds.graph.project.cypher(
        'city_routes', 
        'MATCH (c:City) RETURN id(c) AS id, c.latitude AS latitude, c.longitude AS longitude',
        'MATCH (c1:City)<-[:IN_CITY]-(a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)-[:IN_CITY]->(c2:City) 
        RETURN id(c1) AS source, id(c2) AS target, r.distance AS dist'
    )
    YIELD graphName, nodeCount, relationshipCount
"""


with driver.session() as session:
    run_cypher_query_df(query1)
    result = run_cypher_query_df(query2)
    print(result)
    

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. gds.graph.project.cypher is deprecated. It is replaced by gds.graph.project Cypher projection as an aggregation function.', position=<SummaryInputPosition line=2, column=5, offset=8>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 8, 'line': 2, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "   \n    CALL gds.graph.project.cypher(\n        'city_routes', \n        'MATCH (c:City) RETURN id(c) AS id, c.latitude AS latitude, c.longitude AS longitude',\n        'MATCH (c1:City)<-[:IN_CITY]-(a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)-[:IN_CITY]->(c2:City) \n        RETURN id(c1) AS 

             0     1      2
0  city_routes  3409  46389


Once the projected graph is created, we run A* between two distant island cities: Funafuti (Tuvalu) and Basseterre (Saint Kitts)

In [65]:
with driver.session() as session:
    result = run_cypher_query_df("""
    MATCH (source:City {name: 'Funafuti', country_code: 'TV'}), (target:City {name: 'Basseterre', country_code: 'KN'})
    CALL gds.shortestPath.astar.stream('city_routes', {
        sourceNode: source,
        targetNode: target,
        relationshipWeightProperty: 'dist', 
        latitudeProperty: 'latitude',
        longitudeProperty: 'longitude'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS sourceNodeName,
        gds.util.asNode(targetNode).name AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
        costs
    ORDER BY index
    """)
    for _, row in result.iterrows():
        for i in range(len(row)):
            print(f"{i}: {row[i]}")

0: Funafuti
1: Basseterre
2: 8102.0
3: ['Funafuti', 'Tarawa', 'Majuro Atoll', 'Honolulu', 'Portland', 'New York', 'Basseterre']
4: [0.0, 801.0, 1214.0, 3492.0, 6093.0, 6362.0, 8102.0]


A* returns : 
- 0: The first city
- 1: The second city
- 2: The total length of the shortest path (in miles)
- 3: The list of intermediate cities
- 4: The cumulative cost after each hop (in miles)

This approach demonstrates how Neo4j GDS can use the A* algorithm to compute shortest paths on a custom graph projection derived from the original dataset.

### Centrality Algorithm

#### Betweenness Centrality
Betweenness centrality is a measure of the importance of a node in a network based on how often it appears on the shortest paths between other nodes. A high betweenness centrality indicates that an airport acts as a critical bridge, facilitating connections between many other airports.

To perform the betweenness centrality analysis, we create a projected graph that includes only the Airport nodes and HAS_ROUTE relationships along with the distance property. This projection ensures that the analysis focuses only on airports.

In [66]:
query1 = """   
    CALL gds.graph.exists('airport_relations') YIELD exists
    WITH exists
    WHERE exists
    CALL gds.graph.drop('airport_relations') YIELD graphName
    RETURN graphName
"""

query2 = """   
    CALL gds.graph.project.cypher(
        'airport_relations',
        'MATCH (a:Airport) RETURN id(a) AS id, a.latitude AS latitude, a.longitude AS longitude',
        'MATCH (a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)
        RETURN id(a1) AS source, id(a2) AS target, r.distance AS distance'
    )
    YIELD graphName, nodeCount, relationshipCount
"""

with driver.session() as session:
    run_cypher_query_df(query1)
    result = run_cypher_query_df(query2)
    print(result)

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. gds.graph.project.cypher is deprecated. It is replaced by gds.graph.project Cypher projection as an aggregation function.', position=<SummaryInputPosition line=2, column=5, offset=8>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 8, 'line': 2, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "   \n    CALL gds.graph.project.cypher(\n        'airport_relations',\n        'MATCH (a:Airport) RETURN id(a) AS id, a.latitude AS latitude, a.longitude AS longitude',\n        'MATCH (a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)\n        RETURN id(a1) AS source, id(a2) AS target, r.distance 

                   0     1      2
0  airport_relations  3503  46389


We now compute the betweenness centrality:

In [67]:
with driver.session() as session:
    result = run_cypher_query_df("""
        CALL gds.betweenness.stream('airport_relations')
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).iata AS iata, gds.util.asNode(nodeId).descr AS descr, score
        ORDER BY score DESC
        LIMIT 10
    """)
    print(result)

     0                                        1              2
0  DXB              Dubai International Airport  390958.585096
1  LAX        Los Angeles International Airport  368734.104102
2  CDG                  Paris Charles de Gaulle  365259.650707
3  PEK    Beijing Capital International Airport  340393.582884
4  IST           Istanbul International Airport  339441.886007
5  ORD     Chicago O'Hare International Airport  326830.237013
6  ANC                    Anchorage Ted Stevens  298891.743319
7  FRA                        Frankfurt am Main  285270.758162
8  DFW  Dallas/Fort Worth International Airport  277537.092487
9  AMS               Amsterdam Airport Schiphol  265922.715158


The result obtained was expected. The airports with the highest score are: Dubai, Los Angeles, Paris, Beijing and Istanbul. Istanbul and Dubai are both located between Asia, Europe and Africa. It is often mandatory to fly through them to go from one continent to another. Concerning Paris, Beijing, and Los Angeles, they are the primary gateways for their respective continent, which are the three most developed continents. The conclusion that can be directly inferred from this result is that: If, for some reason, one of these airports were to close, the impact on the global connectivity would be severe. Travelers would need to take other longer, less efficient routes.

An airport with a score of 0 represents small regional airports. This can be verified by changing "DESC" by "ASC" in the request. 

### Visualization 

We can visualize the important airports using Plotly library. You must also have the nbformat library installed, to vizualize inside the notebook.

 Tune the `WHERE` clause to visualize your desired number of airports.

In [68]:
top_airports = run_cypher_query_df("""
CALL gds.betweenness.stream('airport_relations')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS airport, score
WHERE score > 60000 
RETURN airport.iata AS iata, airport.latitude AS lat, airport.longitude AS lon, score
ORDER BY score DESC
""")
import plotly.express as px

fig = px.scatter_geo(
    top_airports,
    lat=1,
    lon=2,
    text=0,
    size=3,
    projection='natural earth',
    title='Airports with Highest Betweenness Centrality'
)
fig.show()


In [69]:
driver.close()
print("\nNeo4j connection closed.")


Neo4j connection closed.


## Conclusions


In this tutorial, we explored Neo4j through geospatial data, using an airport network as a practical example. We went into the essential concepts of graph databases and showed how Neo4j's spatial features and indexing capabilities support efficient geolocation queries. 
By loading, structuring, and querying a dataset of airports, cities, countries, and flight routes, we proved the strengths of Neo4j for handling highly connected data, especially where relationships are as important as the entities themselves. The exercises illustrated both the simplicity of formulating basic graph queries and the expressiveness of more advanced ones that combine spatial reasoning with path traversal. 
We also examined a few of Neo4j’s graph algorithms, showing how they can be easily applied through the Graph Data Science library. Their implementation makes analysis efficient and accessible. 
Overall, the main lesson is that graph databases offer a natural and powerful way to work with complex, relationship-rich datasets, and Neo4j provides an intuitive environment for exploring such data at scale.

## Work distribution

The work was distributed as follows: 
- Emilien DE LA BRASSINNE BONARDAUX: General structure, setup, features, A*, Betweenness centrality
- Yanis GEURTS: General structure, selection of the database, setup of the tutorial, simple query, A*, Betweenness centrality, real-world use case
- Eri VAN DE VYVER: General tutorial structure, Introduction, Advantages and limitations

## Declaration on Generative AI

During the preparation of this work, the authors used chatGPT-5 and Mistral Large to: Grammar and spelling check, improve writing style, and peer review simulation. After using these tools/services, the authors reviewed and edited the content as needed and take full responsibility for the publication’s content.

## Bibliography 

1. Debruyne Christophe. "INFO9016-1 Advanced Databases - Chapter 4: NoSQL Part 2". University of Liège, 2025.
2. Baylor Corydon and Enzo Htet. "What Are the Different Types of Graph Algorithms & When to Use Them?", 29 May 2025, url: https://neo4j.com/blog/graph-data-science/graph-algorithms/. Accessed 14 Nov 2025.
3. Neo4j. Airport Routes Graph Example. GitHub, url: https://github.com/neo4j-graph-examples/airport-routes. Accessed 14 Nov 2025.
4. Dadashzadeh, Ali. "Neo4j vs. SQL: Unlocking the Power of Graph-Based Data Modeling." DEV Community, 19 Feb 2025, url: https://dev.to/ali_dz/neo4j-vs-sql-unlocking-the-power-of-graph-based-data-modeling-33da. Accessed 14 Nov 2025.
5. Kolla, S. (2020). Neo4j Graph Data Science (GDS) library: Advanced analytics on connected data. International Journal of Advanced Research in Engineering and Technology, 11(8), 1077-1086.
6. Neo4j, Inc. "Neo4j Graph Database Documentation." Neo4j, https://neo4j.com/docs. Accessed 21 Nov. 2025.
7. Neo4j, "Next-Gen Application Use Cases", https://neo4j.com/use-cases/. Accessed 21 nov. 2025. 
8. Neo4j. "Gilead Sciences Combats $431 Billion Pharmaceutical Fraud Threat with Neo4j Graph Analytics." Neo4j, 3 Nov. 2025, https://neo4j.com/customer-stories/gilead/.
9. Roemer, Joe. "Improving Patient Outcomes with Graph Algorithms." Neo4j, 24 June 2020, https://neo4j.com/blog/healthcare/improving-patient-outcomes-algorithms-graphconnect/.
10. Neo4j. "U.S. Army." Neo4j, 4 Feb. 2025, https://neo4j.com/customer-stories/u-s-army/.
11. Neo4j. "eBay." Neo4j, 4 Feb. 2025, https://neo4j.com/customer-stories/ebay/.